In [3]:
import pandas as pd
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
import os
from tqdm import tqdm_notebook as tqdm

In [4]:
path = '.\\done\\'

In [60]:
files = [f for f in os.listdir(path) if "for_perl_" in f]

In [61]:
files

['for_perl_mr_id_12.csv', 'for_perl_mr_id_17.csv']

In [77]:
stop_seq = pd.read_csv('.\\data\\stop_seq.csv', sep = ';', encoding = 'cp1251')
stop_count=pd.DataFrame(stop_seq.groupby(['mr_id','rl_id']).agg({'rc_orderby': np.max})).reset_index()

In [78]:
res_stop=pd.DataFrame()
for file in files:
    m = pd.read_csv(path+file, sep = ';', encoding='utf-8')
    m=m[m['len']<100]
    res_stop=res_stop.append(m)
    res_stop=res_stop.sort_values(by=['mr_id','tabelnum','timenav','len'])
    

In [79]:
res_stop=res_stop.reset_index()

In [80]:
res_stop['time']=pd.to_datetime(res_stop['timenav'])
res_stop['hour']=res_stop['time'].dt.hour
res_stop['min']=res_stop['time'].dt.minute
res_stop['sec']=res_stop['time'].dt.second
res_stop['time']=(res_stop['hour'].astype(int))*3600+(res_stop['min'].astype(int))*60+(res_stop['sec'].astype(int))

In [81]:
res_stop.head()

,index,mr_id,rl_id,rc_orderby,ssId,tr_id,timenav,len,geometry,ss_lat,ss_long,tabelnum,number_circle,time,hour,min,sec
0,0,12,47565,30,179,700101007,2020-08-03 05:46:03,95.316912,"LINESTRING (50.125252 53.19819, 50.125252 53.1...",53.198462,50.126609,453,1,20763,5,46,3
1,1,12,47565,31,181,700101007,2020-08-03 05:50:03,80.297151,"LINESTRING (50.123743 53.195042, 50.123743 53....",53.194369,50.123306,453,1,21003,5,50,3
2,2,12,47565,32,1404,700101007,2020-08-03 05:50:03,89.009492,"LINESTRING (50.123743 53.195042, 50.123743 53....",53.194298,50.123250,453,1,21003,5,50,3
3,3,12,47565,31,181,700101007,2020-08-03 05:50:18,24.847958,"LINESTRING (50.123282 53.19459200000001, 50.12...",53.194369,50.123306,453,1,21018,5,50,18
4,4,12,47565,32,1404,700101007,2020-08-03 05:50:18,32.760738,"LINESTRING (50.123282 53.19459200000001, 50.12...",53.194298,50.123250,453,1,21018,5,50,18


In [82]:
#res_stop=res_stop[0:1000]

In [83]:
stop_now=0
number_circle_now=1
time_now=res_stop['time'].loc[0]
rl_id_now=res_stop['rl_id'].loc[0]
tabelnum_now=res_stop['tabelnum'].loc[0]
rc_orderby_now=1
df=pd.DataFrame()
l=0
len_max=res_stop['len'].loc[0]

for index, row in res_stop.iterrows():
    if(row['tabelnum']==tabelnum_now):
        if (row['rc_orderby']==1)&(row['len']<len_max):
            rl_id_now=row['rl_id']
            rc_orderby_now=row['rc_orderby']
            df=df.append(row)
            row['number_circle']=number_circle_now  
            if(row['len']>len_max):
                len_max=row['len']                   
        elif((row['rl_id']==rl_id_now)
            &(row['rc_orderby']==rc_orderby_now+1)
            &(row['len']<len_max)):
            stop_now=row['ssId']
            rc_orderby_now=row['rc_orderby']
            row['number_circle']=number_circle_now
            df=df.append(row)
            stop_num=int(stop_count['rc_orderby'][(stop_count['mr_id']==row['mr_id']) & (stop_count['rl_id']==row['rl_id'])])
            if(row['rc_orderby']==stop_num):
                stop_now=0
                l=0
                number_circle_now=number_circle_now+1
            elif((row['rl_id']==rl_id_now)&(row['rc_orderby']==rc_orderby_now+1)):
                len_max=row['len']  
    else:
        stop_now=0
        l=0
        number_circle_now=1
        tabelnum_now=row['tabelnum']
        if(row['rc_orderby']==1):
            df=df.append(row)

In [74]:
df['ssId'].unique()

array([143.,  42., 204., 144.,  36.,  45., 145., 146., 147., 174., 173.,
       171., 169., 167., 165., 162., 161., 159., 157., 155., 153.,  79.,
        48.,  49.])

In [75]:
df2=df[['len','mr_id','number_circle','ssId','rc_orderby','timenav']]

In [76]:
df2.to_csv('proverka.csv')

In [28]:
df.to_csv('Res2.csv')